In [2]:
# Importing all needed libraries.
from string import ascii_uppercase
from operator import and_
from functools import reduce

In [15]:
class Grammar:
    def __init__(self, Vn, Vt, P, S):
        self.Vn = Vn
        self.Vt = Vt
        self.P = P
        self.S = S
        self.free_uppercase = [letter for letter in ascii_uppercase if letter not in self.Vn]
    
    def __initial_state_S0(self):
        __initial_state_S0 = False
        for i in range(len(self.P)):
            if self.P[i][0] == self.S:
                __initial_state_S0 = True
        if __initial_state_S0:
            self.P.append(('S0', self.S))
            self.S = 'S0'
    
    def __eliminate_terminals(self):
        eliminations = set()
        for production in self.P:
            eliminations.update(set(production[1]).intersection(set(self.Vt)))
        eliminations = list(eliminations)
        for production in self.P:
            if production[1] in self.Vt and production[1] in eliminations:
                eliminations.remove(production[1])
        for element in eliminations:
            self.P.append((self.free_uppercase[0], element))
            self.free_uppercase.pop(0)
        back_mapping = dict()
        for i in range(len(self.P)):
            if self.P[i][1] in self.Vt:
                back_mapping[self.P[i][1]] = self.P[i][0]
        for i in range(len(self.P)):
            for key in back_mapping:
                if key in self.P[i][1] and len(self.P[i][1]) > 1:
                    self.P[i][1] = self.P[i][1].replace(key, back_mapping[key])
    
    def __eliminate2plus__(self):
        while True:
            with_2s = []
            with_2plus = []
            for i in range(len(self.P)):
                if len(self.P[i][1]) == 2:
                    with_2s.append(i)
                elif len(self.P[i][1]) > 2:
                    with_2plus.append(i)
            if len(with_2plus) == 0:
                break
            for i in with_2s:
                for j in with_2plus:
                    if self.P[i][1] in self.P[j][1] and i != j:
                        self.P[j][1] = self.P[j][1].replace(self.P[i][1], self.P[i][0])
            if len(with_2plus) != 0:
                substr = []
                for i in with_2plus:
                    substr.append([])
                    for j in range(len(self.P[i][1]) -1):
                        substr[-1].append(self.P[i][1][j:j+2])
                substr = [set(sub) for sub in substr]
                com_str = list(reduce(and_, substr))
                for com_str in com_str:
                    self.P.append([self.free_uppercase[0], com_str])
                    self.free_uppercase.pop(0)
        return self.P
    
    def to_chomsky(self):
        self.__initial_state_S0()
        self.P = [pair for pair in self.P if pair[1] != 'epsilon']
        self.__eliminate_terminals()
        self.__eliminate2plus__()
        return self.P

In [24]:
dictionary = [['S', 'AC'], ['S', 'bA'], ['S', 'B'], ['S', 'aA'], ['A', 'epsilon'],
              ['A', 'aS'], ['A', 'ABab'], ['B', 'a'], ['B', 'bS'], ['C', 'abC'], 
              ['D', 'AB']]

In [17]:
Vt = ['a', 'b']
Vn = ['S', 'A', 'B', 'C', 'D']

In [18]:
gr = Grammar(Vn, Vt, dictionary, 'S')

In [19]:
gr.to_chomsky()

[['S', 'AB'], ['S', 'c'], ['A', 'a'], ['B', 'b'], ('S0', 'S')]